# Crypto trading strategies
Crypto Investment Strategies

 *  HODLing (Buy and Hold):

      Buying cryptocurrency and holding it long-term, regardless of market fluctuations.
      This strategy is based on the belief that the asset's value will increase over time.
      Best suited for long-term investors with strong conviction in a project's future.

 *  Dollar-Cost Averaging (DCA):

      Investing a fixed amount at regular intervals (e.g., weekly or monthly) regardless of price.
      This reduces the impact of market volatility and avoids emotional decision-making.

 *  Swing Trading:

      Buying low and selling high over short-to-medium time frames
      based on market trends and technical analysis.
      This requires active monitoring and some experience with charts and indicators.

 *  Day Trading:

      Buying and selling crypto assets within the same day to profit from short-term price movements.
      It’s fast-paced, risky, and requires strong technical analysis skills and emotional discipline.

 *  Staking / Yield Farming:

      Locking up crypto assets to earn rewards or interest.
      Staking usually involves securing a blockchain network (like Ethereum 2.0),
      while yield farming involves lending or providing liquidity in DeFi platforms.
      Offers passive income, but carries risks like platform vulnerabilities or token devaluation.

 *  Diversification:

      Spreading investments across multiple cryptocurrencies to reduce overall risk.
      If one asset performs poorly, others might offset the loss.

 *  ICO/IDO Investing (Early Stage Projects):

      Investing in new projects during their Initial Coin Offering (ICO) or Initial DEX Offering (IDO).
      High risk, but potentially high reward. Careful research is essential to avoid scams.

 *  Arbitrage:

      Taking advantage of price differences across different exchanges by buying low on one and selling high on another.
      Requires speed and low transaction fees to be profitable.

In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
# We chose the 1d interval update period
file_path = "btc_1d_data_2018_to_2025.csv"

df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "novandraanugrah/bitcoin-historical-datasets-2018-2024",
  file_path,
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

<ipython-input-1-aa0332f8ad56>:8: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


100%|██████████| 368k/368k [00:00<00:00, 1.75MB/s]


In [2]:
print("First 5 records:", df.head())

First 5 records:     Open time      Open      High       Low     Close        Volume  \
0  2018-01-01  13715.65  13818.55  12750.00  13380.00   8609.915844   
1  2018-01-02  13382.16  15473.49  12890.02  14675.11  20078.092111   
2  2018-01-03  14690.00  15307.56  14150.00  14919.51  15905.667639   
3  2018-01-04  14919.51  15280.00  13918.04  15059.54  21329.649574   
4  2018-01-05  15059.56  17176.24  14600.00  16960.39  23251.491125   

                Close time  Quote asset volume  Number of trades  \
0  2018-01-01 23:59:59.999        1.147997e+08            105595   
1  2018-01-02 23:59:59.999        2.797171e+08            177728   
2  2018-01-03 23:59:59.999        2.361169e+08            162787   
3  2018-01-04 23:59:59.999        3.127816e+08            170310   
4  2018-01-05 23:59:59.999        3.693220e+08            192969   

   Taker buy base asset volume  Taker buy quote asset volume  Ignore  
0                  3961.938946                  5.280975e+07       0  
1    

In [3]:
import pandas as pd

In [4]:
df['Open time'] = pd.to_datetime(df['Open time'])
df.set_index('Open time', inplace=True)
df['Close time'] = pd.to_datetime(df['Close time'])

In [5]:
features_to_transform = ['Open', 'High', 'Low', 'Close', 'Volume']
num_lags = 5
rolling_window = 3

In [6]:
for feature in features_to_transform:
    for lag in range(1, num_lags + 1):
        df[f'{feature}_lag_{lag}'] = df[feature].shift(lag)

In [7]:
df[f'{feature}_rolling_mean_{rolling_window}'] = df[feature].rolling(window=rolling_window).mean()

In [8]:
df[f'{feature}_rolling_std_{rolling_window}'] = df[feature].rolling(window=rolling_window).std()

In [9]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=299adeab36df3dba3a9c9d2da96bf001d8ec8385e2bb7dcbb63b2053bc4e971d
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [10]:
import ta

In [11]:
df['rsi'] = ta.momentum.RSIIndicator(df['Close']).rsi()
df['macd'] = ta.trend.MACD(df['Close']).macd()
df['ema_12'] = ta.trend.EMAIndicator(df['Close'], window=12).ema_indicator()
df['ema_26'] = ta.trend.EMAIndicator(df['Close'], window=26).ema_indicator()

In [12]:
future_shift = 1  # predict 1 day ahead
df['future_close'] = df['Close'].shift(-future_shift)

In [13]:
import numpy as np

In [14]:
# df['target'] = np.where(df['future_close'] > df['Close'] * 1.01, 2,
#                         np.where(df['future_close'] < df['Close'] * 0.95, 1, 0)) # 10 percent increase / 5 decrease

In [15]:
strategy = 3  # Change this to test different strategies (1 to 8)

if strategy == 1:  # HODLing
    df['target'] = np.where(df['future_close'] < df['Close'] * 0.7, 2,  # Buy if it dropped 30%
                     np.where(df['future_close'] > df['Close'] * 1.7, 1, 0))  # Sell if we have 70% increase

elif strategy == 2:  # Dollar-Cost Averaging (DCA)
    df['target'] = 2  # Always Buy

elif strategy == 3:  # Swing Trading
    # Buy if future gain >10%, Sell if expected drop >10%
    df['target'] = np.where(df['future_close'] > df['Close'] * 1.10, 2,
                     np.where(df['future_close'] < df['Close'] * 0.90, 1, 0))

elif strategy == 4:  # Day Trading
    # Buy/Sell on small moves (+3%, -3%)
    df['target'] = np.where(df['future_close'] > df['Close'] * 1.03, 2,
                     np.where(df['future_close'] < df['Close'] * 0.97, 1, 0))

elif strategy == 5:  # Staking / Yield Farming
    # Buy low-volatility coins, Sell high-volatility
    df['volatility'] = (df['High'] - df['Low']) / df['Close']
    df['target'] = np.where(df['volatility'] < 0.02, 2,
                     np.where(df['volatility'] > 0.08, 1, 0))

elif strategy == 6:  # Diversification
    # Buy underexposed coins, Sell overexposed ones
    avg_close = df['Close'].mean()
    df['target'] = np.where(df['Close'] < avg_close * 0.8, 2,
                     np.where(df['Close'] > avg_close * 1.2, 1, 0))

elif strategy == 7:  # ICO/IDO
    # Buy if future close is massively higher (2x), Sell if big loss
    df['target'] = np.where(df['future_close'] > df['Close'] * 2.0, 2,
                     np.where(df['future_close'] < df['Close'] * 0.8, 1, 0))

elif strategy == 8:  # Arbitrage
    # Buy if price much lower than avg, Sell if much higher
    df['avg_price'] = df[['Open', 'High', 'Low', 'Close']].mean(axis=1)
    df['target'] = np.where(df['Close'] < df['avg_price'] * 0.95, 2,
                     np.where(df['Close'] > df['avg_price'] * 1.05, 1, 0))

else:
    raise ValueError("Unknown strategy.")


In [16]:
# df['target'] = -1 # Hold
# df.loc[df['future_close'] > df['Close'] * (1 + 0.001), 'target'] = 1  # Buy
# df.loc[df['future_close'] < df['Close'] * (1 - 0.001), 'target'] = 0  # Sell

In [17]:
df.dropna(inplace=True)

In [18]:
df[['Close', 'future_close', 'target']]

,Close,future_close,target
Open time,,,
2018-01-26,11089.00,11491.00,0
2018-01-27,11491.00,11879.95,0
2018-01-28,11879.95,11251.00,0
2018-01-29,11251.00,10237.51,0
2018-01-30,10237.51,10285.10,0
...,...,...,...
2025-05-16,104234.77,102822.55,0
2025-05-17,102822.55,103289.62,0
2025-05-18,103289.62,105140.00,0


In [19]:
train_size = int(len(df) * 0.8)
train_df = df.iloc[:train_size]
test_df = df.iloc[train_size:]

In [20]:
X_train = train_df.drop(['future_close', 'target', 'Close time'], axis=1)
y_train = train_df['target']
X_test = test_df.drop(['future_close', 'target', 'Close time'], axis=1)
y_test = test_df['target']

In [21]:
# We resample the dataset to force the data to learn and not default to the majority
from imblearn.over_sampling import SMOTE

#sm = SMOTE(random_state=42)
#X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

In [22]:
X_train = train_df.drop(['future_close', 'target', 'Close time'], axis=1)
y_train = train_df['target']
X_test = test_df.drop(['future_close', 'target', 'Close time'], axis=1)
y_test = test_df['target']

In [23]:
X_train.head(5)

,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore,...,Volume_lag_2,Volume_lag_3,Volume_lag_4,Volume_lag_5,Volume_rolling_mean_3,Volume_rolling_std_3,rsi,macd,ema_12,ema_26
Open time,,,,,,,,,,,,,,,,,,,,,
2018-01-26,11184.70,11643.00,10311.15,11089.00,33056.871960,3.604825e+08,314720,15330.789553,1.673290e+08,0,...,27158.587762,37473.922552,43752.606791,41379.773426,27018.471302,6109.664022,39.612390,-794.995978,11701.843088,12496.839066
2018-01-27,11089.00,11650.00,10842.69,11491.00,18860.768345,2.126632e+08,207478,9208.937963,1.039359e+08,0,...,20839.954183,27158.587762,37473.922552,43752.606791,24252.531496,7688.732171,42.915669,-752.926779,11669.405690,12422.332469
2018-01-28,11499.98,12244.00,11408.00,11879.95,16887.339524,2.002116e+08,206247,7342.786997,8.701101e+07,0,...,33056.871960,20839.954183,27158.587762,37473.922552,22934.993276,8821.163551,45.993842,-680.358867,11701.797122,12382.155989
2018-01-29,11879.95,11975.02,11139.55,11251.00,14170.377538,1.621102e+08,169199,6016.496871,6.881997e+07,0,...,18860.768345,33056.871960,20839.954183,27158.587762,16639.495136,2354.997157,42.045600,-665.922938,11632.443719,12298.366657
2018-01-30,11250.11,11308.42,9900.00,10237.51,25554.372946,2.689783e+08,259212,11388.096419,1.198643e+08,0,...,16887.339524,18860.768345,33056.871960,20839.954183,18870.696669,5945.512344,36.594241,-727.872077,11417.838531,12145.710608


In [24]:
import xgboost as xgb

In [25]:
import numpy as np

In [26]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=3,
    eval_metric='mlogloss',
    use_label_encoder=False,
    verbosity=0
)

param_grid = {
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.001, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='f1_weighted',
    cv=3,
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
print("Best parameters:", grid_search.best_params_)


Fitting 3 folds for each of 144 candidates, totalling 432 fits
Best parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.8}


In [27]:
# y_pred = best_model.predict(X_test)

In [28]:
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

In [29]:
from sklearn.metrics import classification_report

In [30]:
print("Classification Report:\n", classification_report(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00       532
           2       0.00      0.00      0.00         3

    accuracy                           0.99       535
   macro avg       0.50      0.50      0.50       535
weighted avg       0.99      0.99      0.99       535



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [31]:
print("Train target distribution:\n", y_train.value_counts(normalize=True))
print("Test target distribution:\n", y_test.value_counts(normalize=True))

Train target distribution:
 target
0    0.974731
2    0.014038
1    0.011231
Name: proportion, dtype: float64
Test target distribution:
 target
0    0.994393
2    0.005607
Name: proportion, dtype: float64


In [32]:
unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))

{np.int64(0): np.int64(2083), np.int64(1): np.int64(24), np.int64(2): np.int64(30)}


In [33]:
# Базирано на трудот [Madan et al.]

In [34]:
# Momentum, Bollinger Bands, ADX
# Додавање дополнителни индикатори
df['momentum'] = ta.momentum.ROCIndicator(df['Close']).roc()
bb = ta.volatility.BollingerBands(df['Close'])
df['bb_high'] = bb.bollinger_hband()
df['bb_low'] = bb.bollinger_lband()
df['adx'] = ta.trend.ADXIndicator(df['High'], df['Low'], df['Close']).adx()

# RSI, MACD, EMA (ги имаш веќе)
df['rsi'] = ta.momentum.RSIIndicator(df['Close']).rsi()
df['macd'] = ta.trend.MACD(df['Close']).macd()
df['ema_12'] = ta.trend.EMAIndicator(df['Close'], window=12).ema_indicator()
df['ema_26'] = ta.trend.EMAIndicator(df['Close'], window=26).ema_indicator()

# Target
df['future_close'] = df['Close'].shift(-future_shift)
df['target'] = np.where(df['future_close'] > df['Close'] * 1.01, 2,
                        np.where(df['future_close'] < df['Close'] * 0.95, 1, 0))

In [35]:
#Генерирање на текстуални промптови за LLM класификација
def generate_prompt(row):
    return (
        f"On {row.name.date()}, Bitcoin closed at {row['Close']:.2f}, "
        f"high: {row['High']:.2f}, low: {row['Low']:.2f}, volume: {row['Volume']:.2f}. "
        f"RSI: {row['rsi']:.2f}, MACD: {row['macd']:.2f}, Momentum: {row['momentum']:.2f}, "
        f"EMA12: {row['ema_12']:.2f}, EMA26: {row['ema_26']:.2f}, ADX: {row['adx']:.2f}. "
        f"Classify the next day's movement as: increase, decrease, or neutral."
    )

#последни редици за пример
sample_df = df.dropna().tail(5)
sample_prompts = sample_df.apply(generate_prompt, axis=1)

for prompt in sample_prompts:
    print("\n=== Prompt ===\n", prompt)


=== Prompt ===
 On 2025-05-15, Bitcoin closed at 103059.60, high: 103866.51, low: 102968.16, volume: 866.47. RSI: 68.44, MACD: 3910.49, Momentum: 6.83, EMA12: 100725.03, EMA26: 96814.54, ADX: 31.88. Classify the next day's movement as: increase, decrease, or neutral.

=== Prompt ===
 On 2025-05-16, Bitcoin closed at 104234.77, high: 104279.07, low: 103467.91, volume: 1800.88. RSI: 70.42, MACD: 3900.81, Momentum: 8.39, EMA12: 101264.99, EMA26: 97364.18, ADX: 32.44. Classify the next day's movement as: increase, decrease, or neutral.

=== Prompt ===
 On 2025-05-17, Bitcoin closed at 102822.55, high: 103539.69, low: 102612.50, volume: 1636.40. RSI: 65.13, MACD: 3736.11, Momentum: 9.65, EMA12: 101504.62, EMA26: 97768.51, ADX: 32.48. Classify the next day's movement as: increase, decrease, or neutral.

=== Prompt ===
 On 2025-05-18, Bitcoin closed at 103289.62, high: 103409.98, low: 103105.09, volume: 516.83. RSI: 66.04, MACD: 3601.75, Momentum: 9.49, EMA12: 101779.23, EMA26: 98177.48, ADX

In [36]:
pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.3/683.3 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.78.1
    Uninstalling openai-1.78.1:
      Successfully uninstalled openai-1.78.1
